In [2]:
import pandas as pd

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader

In [4]:
df = pd.read_csv("100_Unique_QA_Dataset.csv")

In [5]:
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


# Tokenize 

In [6]:
def tokenize(text):
    text = text.lower()
    text = text.replace('?', '')
    text = text.replace("'", '')
    return text.split()

In [7]:
vocab = {'<UNK>': 0}

In [8]:
def build_vocab(row):
  tokenized_q = tokenize(row['question'])
  tokenized_ans = tokenize(row['answer'])

  merged_tokens = tokenized_q + tokenized_ans

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)

In [9]:
df.apply(build_vocab, axis = 1)
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [10]:
len(vocab)

324

In [11]:
def sent_to_indices(sent, vocab):

    indexed_sent = []

    for word in tokenize(sent):

        if word in vocab:
            indexed_sent.append(vocab[word])
        else:
            indexed_sent.append(vocab['<UNK>'])
            
    return indexed_sent


In [12]:
class qnaData(Dataset): 

    def __init__(self, df, vocab):
        self.df = df
        self.vocab = vocab

    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
        
        indexed_question = sent_to_indices(self.df.loc[index]['question'], self.vocab)
        indexed_answer = sent_to_indices(self.df.loc[index]['answer'], self.vocab)

        return torch.tensor(indexed_question), torch.tensor(indexed_answer)


In [13]:
dataset = qnaData(df, vocab)

In [14]:
trainLoader = DataLoader(dataset=dataset, batch_size=1, shuffle=True)

In [15]:
import torch.nn as nn
import torch.optim as optim

In [18]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.Wx = nn.Parameter(torch.randn(hidden_size, embedding_dim) * 0.01)
        self.Wh = nn.Parameter(torch.randn(hidden_size, hidden_size) * 0.01)
        self.bh = nn.Parameter(torch.zeros(hidden_size))

        self.Wy = nn.Parameter(torch.randn(vocab_size, hidden_size) * 0.01)
        self.by = nn.Parameter(torch.zeros(vocab_size))

    def forward(self, X):

        batch_size, seq_len = X.size()
        h = torch.zeros(batch_size, self.hidden_size, device=X.device)  # initial hidden state

        X_embedded = self.embedding(X)
        for t in range(seq_len):
            x_t = X_embedded[:, t, :] 
            h = torch.tanh(x_t @ self.Wx.T + h @ self.Wh.T + self.bh)

        y = h @ self.Wy.T + self.by
        return y

In [19]:
learning_rate = 0.001
epochs = 20
vocab_size = len(vocab)
embedding_dim = 64
hidden_size = 128
model = RNN(vocab_size, embedding_dim, hidden_size)

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [21]:
for epoch in range(epochs):
    total_loss = 0
    for X, y in trainLoader:

        optimizer.zero_grad()

        y_pred = model(X)
        y = y.view(-1).long()
        loss = criterion(y_pred, y)

        loss.backward()

        optimizer.step()

        total_loss += loss.item()
    print(f"Loss at epoch {epoch + 1} = {total_loss:4f}")

Loss at epoch 1 = 520.649017
Loss at epoch 2 = 473.251317
Loss at epoch 3 = 389.416943
Loss at epoch 4 = 312.346658
Loss at epoch 5 = 238.644241
Loss at epoch 6 = 168.468743
Loss at epoch 7 = 110.854190
Loss at epoch 8 = 70.297237
Loss at epoch 9 = 46.290704
Loss at epoch 10 = 31.882951
Loss at epoch 11 = 22.866022
Loss at epoch 12 = 17.574312
Loss at epoch 13 = 14.402855
Loss at epoch 14 = 11.206402
Loss at epoch 15 = 9.736711
Loss at epoch 16 = 10.131553
Loss at epoch 17 = 7.171002
Loss at epoch 18 = 5.680758
Loss at epoch 19 = 4.738938
Loss at epoch 20 = 4.068633


In [22]:
def predict(model, question, threshold = 0.5):
    
    indiced = sent_to_indices(question, vocab)

    indiced_tensor = torch.tensor(indiced).unsqueeze(0)    
    
    y_pred = model(indiced_tensor)
    # These are logits we need to convert it to probabilities

    # we will pass the outputs through softmax to convert them to probabilities
    y_pred = nn.functional.softmax(y_pred, dim = 1)
    
    value, idx = torch.max(y_pred, dim = 1)
    
    if value < threshold:
        print("I don't know")
    else:
        print(list(vocab.keys())[idx])



In [23]:
predict(model, "What is the capital of France ?")

paris


In [24]:
predict(model, "To Kill a Mockingbird")

harper-lee
